### Introducción
El objetivo de este notebook es mostrar como utilizar `pandas` para hacer operaciones de resampleado (`resample`) y combinado (`merge`) con distintos dataframes.

### Dataset
El dataset sobre el cual se trabajará consta de varios ficheros. A continuación se describirán los utilizados,
* **all_lite.txt**, se trata de una pequeña muestra de un fichero que contiene datos horarios sobre los precios de distintos tipos de combustibles medidos en 10233 gasolineras de toda España. Cada tiene además datos propios de cada gasolinera (grupo al que pertenece, dirección, horario...) y la variable target: *precio_sig* que es el precio del diesel en la siguiente hora. Para cada gasolinera se proporciona una serie temporal de como máximo 641 registros (ó lo que es lo mismo 641 horas≈27 días). El fichero original es de giga y medio de tamaño con lo que manejarlo en local es posible pero pesado. Se proporciona un versión reducida para el estudio previo de los datos.
* **eur_usd.txt**, contiene el histórico del precio de cierre diario para el tipo de cambio EUR/USD.
* **brent.txt**, contiene el histórico de distintos parámetros diarios de mercado relativos al barril de petróleo Brent (por ejemplo precio de apertura, máximo, mínimo, de cierre...)

### Manos a la obra
Cargaremos primeramente el fichero **all_lite.txt**

In [ ]:
# Directorio padre
DATASET_DIR=".\\datasets\\"
# Subdirectorio al que llamamos diesel
PRECIO_DIESEL_DATASET_DIR="diesel"
# Fichero a cargar
PRECIO_DIESEL_FILENAME="all_lite.txt"

Esto lo haremos en un dataframe de `pandas` y su método `read_csv`. Aquí tenemos dos opciones:
* Dejar que Python infiera los `dtypes` esto se conseguiría no especificando este parámetro al método `read_csv`. Es más cómodo pero a cambio Python considerará todos aquellos tipos de dato numéricos como `float64` y aquellos que no sean estrictamente numéricos como `object` lo que se traducirá en una mayor consumo de RAM.
* Explorar previamente la versión lite del fichero a cargar para estudiar los valores entre los que se mueve cada campo, segúramente no todos ellos precisen de un tipo `float64` y para otros el tipo `object` no será el más óptimo. Una vez hecho el estudio, especificaremos mediante el campo `dtype` un diccionario con el tipo a utilizar para cargar cada campo.

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
PRECIOS_DIESEL_PATH = os.path.join(DATASET_DIR, PRECIO_DIESEL_DATASET_DIR, PRECIO_DIESEL_FILENAME)

In [ ]:
precios_diesel_df = pd.read_csv(filepath_or_buffer=PRECIOS_DIESEL_PATH,\
                                    dtype={"ideess":np.int16,"hora":np.uint8,"remision":"category","precio":np.float16,\
                                           "precio_premium":np.float16,"precio_gasolina_95":np.float16,"precio_ant":np.float16,\
                                           "precio_sig":np.float16,"diff_premium":np.float16,"diff_ant":np.float16,\
                                           "ind_cambio":np.float16,"ind_diff":np.float16,"ind_diff_sig":np.float16,"diff_sig":np.float16,\
                                           "grupo":"category","ind_24h":"category","codpos":np.uint16,"idccaa":np.uint8,"idmunicipio":np.uint16,\
                                           "idprovincia":np.uint8,"latitud":np.float16,"longitud":np.float16,"margen":"category",\
                                           "ind_intradia":"category","max_cambios":np.float16,"min_cambios":np.float16,"avg_cambios":np.float16},\
                                    sep='|', 
                                    parse_dates=['dia','fecha'],
                                    infer_datetime_format=True)

Después de ejecutar un [`pandas profiling`](https://github.com/pandas-profiling/pandas-profiling) sobre el dataset y dado que lo que tendremos en cuenta para el ejemplo se quiere mostrar son datos que varian en el tiempo, en base a esto realizamos un primer descarte de features:

In [ ]:
drop_list = ['ccaa','codpos','diff_ant','diff_sig','direccion','fecha','grupo',\
             'horario','idccaa','ideess','idmunicipio','idprovincia','ind_cambio',\
             'ind_cambio_sig','ind_intradia','latitud','localidad','longitud','margen',\
             'max_cambios','min_cambios','municipio','precio_ant','provincia','remision','rotulo']

precios_diesel_df_clean = precios_diesel_df.drop(columns=drop_list,inplace=False)

In [ ]:
precios_diesel_df_clean.head()

In [ ]:
# Con la siguiente operación, simplemente reordenamos las columnas para que
# la variable target (precio_sig) ocupe la última posición
precios_diesel_df_clean = precios_diesel_df_clean[[c for c in precios_diesel_df_clean if c not in ['precio_sig']] + ['precio_sig']]

In [ ]:
precios_diesel_df_clean.head()

### Tipo de cambio EUR/USD
Es el momento de cargar en un dataframe de `pandas` el fichero de tipos de cambio. Lo que queremos hacer es enriquecer los datos propios de los precios del diesel que acabamos de cargar mediante el fichero all_lite.txt, incluyendo el tipo de cambio EUR/USD que hemos obtenido de otra fuente.

In [ ]:
# Fichero a cargar
PRECIO_EURUSD_FILENAME="eur_usd.txt"

In [ ]:
# Se construye la ruta de la cual se cargarán los precios del tipo de cambio
PRECIOS_EURUSD_PATH = os.path.join(DATASET_DIR, PRECIO_DIESEL_DATASET_DIR, PRECIO_EURUSD_FILENAME)

In [ ]:
precios_eurusd_df = pd.read_csv(filepath_or_buffer=PRECIOS_EURUSD_PATH,\
                                dtype={"eur_usd":np.float32},\
                                sep='|',\
                                parse_dates=['fecha'],\
                                infer_datetime_format=True)

Hay que tener en cuenta que los precios del fichero **all_lite.txt** son diarios, incluyendo fines de semana y festivos. Entonces si queremos emparejar cada uno de los días disponibles con el correspondiente tipo de cambio del fichero **eur_usd.txt**, tendremos que cerciorarnos de proveer precio todos los días para este último...

In [ ]:
# Echemos un vistazo rápido
precios_eurusd_df.head(n=7)

¡Vaya! vemos que existe algún hueco.
Vamos a rellenar los huecos ocasionados por fines de semana y festivos repitiendo el último tipo de cambio disponible (por ejemplo tomando el precio del viernes y aplicándolo al sábado y domingo siguientes). Esto una entre las muchas posibilidades que ofrece el método `resample`.

In [ ]:
# En el caso de que no hayamos convertido nuestro campo de tipo fecha en el método read_csv
# tendremos que asegurarnos de que el campo según el cual queremos indexar sea de tipo fecha
# (bastaría con descomentar la siguiente líneas) para realizar el resample sobre el.

#precios_eurusd_df["fecha"] = pd.to_datetime(precios_eurusd_df["fecha"],format="%Y/%m/%d")
precios_eurusd_df = precios_eurusd_df.sort_values(["fecha"], ascending=[True])

In [ ]:
precios_eurusd_df.head()

In [ ]:
# Para poder hacer el resample, la feature fecha será establecida como índice del dataframe
precios_eurusd_df.set_index("fecha",inplace=True)

In [ ]:
# Ahora el dataframe está indexado por la feature fecha
precios_eurusd_df.head(n=7)

In [ ]:
# Ahora ya podemos hacer el resampling, "D" indica que será efectuado a nivel diario
# (existen diferentes tipos de reglas para especificar sobre qué base temporal)
# Aquí hay una lista de offsets http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
# ahora elegiremos el método de rellenado de huecos, en este caso ffill() llenará los huecos diarios en 
# nuestra serie y los rellenará con el último valor disponible.
precios_eurusd_df = precios_eurusd_df.resample("D").ffill().reset_index()

In [ ]:
# Comprobamos algún hueco para ver si se ha rellenado
precios_eurusd_df.head(n=7)

Si vamos a incluir el tipo de cambio en nuestro dataset cargado a partir de **all_lite.txt**, hemos de tener en cuenta que a la hora de hacer una predicción sobre el precio de la hora siguiente tendremos que incluir también la feature tipo de cambio, la única manera de poder hacer esto es considerar para cada día de **all_lite.txt** el tipo de cambio del día anterior: es decir el día 17 de enero sabremos cual fue el tipo cambio final para el día 16 de enero.

Por ello vamos a crear una nueva feature en el dataset del tipo de cambio que indique que la fecha en la que aplicaremos cada tipo de cambio, será un día después de la fecha de cierre del mismo.

In [ ]:
# TimeDelta nos permite declarar escalares medidos en distintas unidades de tiempo
# de manera que podamos realizar operaciones con fechas
precios_eurusd_df["fecha_join"] = precios_eurusd_df["fecha"] + pd.to_timedelta(1,unit="d")

In [ ]:
# Ahora tenemos la fecha efectiva para cada tipo de cambio EURUSD
# pero también tenemos la fecha con la que emparejaremos cada registro
# del all_lite.txt
precios_eurusd_df.tail(3)

### Precios barril de Brent
Ahora cargaremos en un dataframe de `pandas` el histórico de precios del barril de Brent (del fichero ** brent.txt **). El procedimiento será prácticamente idéntico al seguido con el tipo de cambio, con la diferencia de que para el barril de Brent disponemos de más features.

In [ ]:
# Fichero a cargar
PRECIO_BRENT_FILENAME="brent.txt"

In [ ]:
# Se construye la ruta de la cual se cargarán los precios del barril de Brent
PRECIOS_BRENT_PATH = os.path.join(DATASET_DIR, PRECIO_DIESEL_DATASET_DIR, PRECIO_BRENT_FILENAME)

De nuevo, le damos ya los `dtype` a Python de manera que no tenga que inferirlos. Préviamente a esto hemos revisado el pandas profiling para poder adecuar los `dtype` a cada feature.

In [ ]:
precios_brent_df = pd.read_csv(filepath_or_buffer=PRECIOS_BRENT_PATH,\
                                    dtype={"open":np.float16,"high":np.float16,"low":np.float16,"settle":np.float16,\
                                           "change":np.float16,"wave":np.float16,"volume":np.float32,\
                                           "prev_day_open_interest":np.float32,"efp_volume":np.float32,"efs_volume":np.float16,\
                                           "block_volume":np.float16},\
                                    sep='|',\
                                    parse_dates=['fecha'],\
                                    infer_datetime_format=True)

Formateamos y establecemos la fecha como índice del dataframe de manera que posteriormente podamos efectuar un `resample` de forma similar a como se hizo en el caso del fichero de tipos de cambio.

In [ ]:
# En el caso de que no hayamos convertido nuestro campo de tipo fecha en el método read_csv
# tendremos que asegurarnos de que el campo según el cual queremos indexar sea de tipo fecha
# (bastaría con descomentar la siguiente línea)
#precios_brent_df["fecha"] = pd.to_datetime(precios_brent_df["fecha"], format="%Y/%m/%d")
# para realizar el resampling sobre el

precios_brent_df = precios_brent_df.sort_values(by=["fecha"], ascending=[True])
precios_brent_df.set_index("fecha", inplace=True)

In [ ]:
precios_brent_df.head()

In [ ]:
precios_brent_df = precios_brent_df.resample('D').ffill().reset_index()

In [ ]:
precios_brent_df.head()

Ahora obtendremos la fecha en la que asociaremos cada precio del barril Brent a su correspondiente registro en **all_lite.txt** (de la misma manera que hicimos con el tipo de cambio EURUSD)

In [ ]:
precios_brent_df["fecha_join"] = precios_brent_df["fecha"] + pd.to_timedelta(1, unit='d')

In [ ]:
precios_brent_df.head()

### Fusionando (merge) los dataframe `precios_diesel_df_clean`, `precios_eurusd_df` y `precios_brent_df`
Ahora es cuando uniremos los tres dataframe en base a los ajustes de fecha que hemos hecho en `precios_eurusd_df` y `precios_brent_df` apoyándonos en las features `fecha_join` que hemos ido creando con la fecha que utilizaremos para unir ambos dataframes con `precios_diesel_df_clean`.

In [ ]:
# Lo primero será unir precios_diesel_df_series con precios_brent_df
precios_diesel_brent_df = precios_diesel_df_clean.merge(right=precios_brent_df, how='left', left_on=["dia"], right_on=["fecha_join"])

In [ ]:
precios_diesel_brent_df.head()

In [ ]:
precios_diesel_brent_df.columns

In [ ]:
# Ahora, con el dataframe resultante, repetiremos el join con precios_eurusd_df
precios_diesel_brent_eurusd_df = precios_diesel_brent_df.merge(right=precios_eurusd_df, how='left', left_on=["fecha_join"], right_on=["fecha_join"])

In [ ]:
precios_eurusd_df.head()

In [ ]:
precios_diesel_brent_eurusd_df.columns

Ahora que ya tenemos los tres dataframe unidos, aprovechamos para visualizar el tipo de cambio en combinación con el precio del diesel y el settle del Brent.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
columns = ["eur_usd","precio","settle"]

plt.figure()
for idx,col in enumerate(columns):
    plt.subplot(len(columns),1,idx+1)
    plt.plot(precios_diesel_brent_eurusd_df[col].values)
    plt.title(col, y=0.7, loc='left')
plt.xlabel("tiempo (horas)")
plt.tight_layout()
plt.show()

In [ ]:
for idx,col in enumerate(columns):
    print(str(idx) + " " + col)

Finalmente daremos un par de retoques al dataframe final situando la columna label al final del mismo y deshaciéndonos de aquellas features auxiliares que se han utilizado para unir los dataframes.

In [ ]:
drop_list = ['dia','hora','fecha_x','fecha_y','fecha_join','ind_24h']

precios_diesel_brent_eurusd_df.drop(columns=drop_list,inplace=True)

In [ ]:
precios_diesel_brent_eurusd_df = precios_diesel_brent_eurusd_df[[c for c in precios_diesel_brent_eurusd_df if c not in ['precio_sig']] + ['precio_sig']]

Este es el aspecto del dataframe resultante

In [ ]:
precios_diesel_brent_eurusd_df.head()